In [4]:
from bs4 import BeautifulSoup
import requests
import csv

In [5]:
#def getPsychData(city, state):

state = 'NY'
city = 'Brooklyn'

url = 'https://www.psychologytoday.com/us/therapists/' + state.lower() + '/' + city.lower()
# header added after running requests.get(url) by itself returned a 403 status code
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15'}
mainpagehtml = requests.get(url,headers=headers)

In [6]:
# turn html into BeautifulSoup object
soup = BeautifulSoup(mainpagehtml.content,'html.parser')
mainpage = soup.body

# site uses this div class for therapist listings in results. ResultSet object.
therapist_results = soup.find_all('div',{'class': 'result-row normal-result row'})

In [149]:
# therapist info is in span tags; itemprop attribute gives label for info (eg job title, etc)
spans = therapist_results[1].find_all('span')
spanitemprops = []
for span in spans:
    if span.get('itemprop'): spanitemprops.append(span.get('itemprop'))

therapists = []
for i in range(0,len(therapist_results)):
    propertieslist = therapist_results[i]('span',{'itemprop':spanitemprops})
    therapist_info = {}
    for prop in propertieslist:
        if prop.get_text(): 
# lots of buttons/subtags that have '\n\n\n' as get_text(), some itemprop attributes (eg city) repeat within listings
            if '\n' in prop.get_text(): pass
            elif prop.get('itemprop') in therapist_info.keys(): pass
            else: therapist_info[prop.get('itemprop')] = prop.get_text()
    therapist_info['phone'] = therapist_results[i].get('data-phone')
    therapists.append(therapist_info)
therapists

[{'name': 'Jenny Marion',
  'jobTitle': 'Psychologist',
  'addressLocality': 'Brooklyn,',
  'addressRegion': 'New York',
  'postalCode': '11217',
  'phone': '(518) 720-6285'},
 {'name': 'Terry Cramer',
  'jobTitle': 'Clinical Social Work/Therapist',
  'addressLocality': 'Brooklyn,',
  'addressRegion': 'New York',
  'postalCode': '11215',
  'phone': '(917) 791-1226'},
 {'name': 'Joanne Maude Heith',
  'jobTitle': 'Clinical Social Work/Therapist',
  'addressLocality': 'Brooklyn,',
  'addressRegion': 'New York',
  'postalCode': '11201',
  'phone': '(347) 970-7620'},
 {'name': 'Erica Gaeta',
  'jobTitle': 'Clinical Social Work/Therapist',
  'addressLocality': 'Brooklyn,',
  'addressRegion': 'New York',
  'postalCode': '11222',
  'phone': '(917) 924-6415'},
 {'name': 'Kate Klein',
  'jobTitle': 'Counselor',
  'addressLocality': 'Brooklyn,',
  'addressRegion': 'New York',
  'postalCode': '11242',
  'phone': '(914) 873-1250'},
 {'name': 'Stephanie Rozen',
  'jobTitle': 'Clinical Social Work/T

In [17]:
# now grab links to profiles so we can get actual info including contact info
therapist_site_ids = []
for t in therapist_results:
    if t.get('data-new-clients')=='1':
        therapist_site_ids.append(t.get('data-profid'))

In [162]:
profid = therapist_site_ids[6]
#for profid in therapist_site_ids:
url = "https://www.psychologytoday.com/profile/" + profid
pagehtml = requests.get(url,headers=headers)
page = BeautifulSoup(pagehtml.content,'html.parser')
page.find('div',{'class':'container main-content profile'})

<div class="container main-content profile" data-email-url="https://www.psychologytoday.com/us/profile/348164/contact" data-new-clients="1" data-phone="(646) 832-2513" data-prof-name="Dianne Ghiraj-Castor" data-profid="348164" data-profile-image="https://cdn4.sussexdirectories.com/rms/rms_photos/sized/64/81/348164-1087166-2_320x400.jpg?pu=1507085400" data-profile-url="https://www.psychologytoday.com/us/therapists/dianne-ghiraj-castor-brooklyn-ny/348164" data-use-company-name="" id="profileContainer" itemscope="" itemtype="http://schema.org/Person">
<div class="row" id="profHdr">
<div class="col-12 col-sm-12 hidden-md-up profile-title-container">
<h1 itemprop="name">
                    Dianne Ghiraj-Castor</h1>
<div class="profile-title">
<h2> <span class="nowrap">
<span data-glossary-url="https://www.psychologytoday.com/us/therapists/glossary/1302" data-ui-type="glossary">Counselor</span><span>, </span> </span>
<span class="nowrap">
<span data-glossary-url="https://www.psychologytoday

In [181]:
def getInfo(pg,insdict):
    gotInfo = {}
    
    gotInfo['name'] = cleanString(pg.find('h1',{'itemprop':'name'}).get_text())
    gotInfo['phone'] = pg.find('div',{'class':'container main-content profile'}).find('a').get('href').split('+')[1]
    gotInfo['address'] = pg.find('div',{'class':"address-data"}).get('data-address-rank-1')
    gotInfo['title'] = simplifyTitle(pg.find('div',{'class':'modal-subtitle'}).get_text())
    gotInfo['remote'] = findRemote(pg)
    gotInfo['insurance'] = findInsurance(pg,insdict)
    
    
    return gotInfo

getInfo(page,insurancedict)

{'name': 'Dianne Ghiraj-Castor',
 'phone': '1-646-832-2513',
 'address': '223 Jay Street, Brooklyn, New York, 11201',
 'title': 'Therapist',
 'remote': 'Online Counseling',
 'insurance': {'Out of Network': False,
  'Any Insurance': False,
  'Cigna': True,
  'BlueCross and BlueShield': True,
  'Aetna': False,
  'Empire BlueCross BlueShield': True,
  '1199SEIU': False,
  'AMERIGROUP': False,
  'APS Healthcare': False,
  'Affinity Health Plan': False,
  'AllWays Health Partners': False,
  'Alliance': False,
  'AmeriHealth': False,
  'American Behavioral': False,
  'Anthem': True,
  'Beacon': False,
  'Beech Street': False,
  'Behavioral Health Systems': False,
  'Blue Care Network': False,
  'Blue Cross': False,
  'Blue Shield': False,
  'BlueCross BlueShield of Western New York': False,
  'BlueShield of Northeastern New York': False,
  'CCN': False,
  'CDPHP': False,
  "CHIP | Children's Program": False,
  'CHIPA': False,
  'CareConnect Health Plan': False,
  'CareFirst': False,
  'CareS

In [153]:
title = page.find('div',{'class':"modal-subtitle"}).get_text()

def simplifyTitle(string):
    if 'Psychiatrist' in string: return "Psychiatrist"
    elif 'Psychologist' in string: return 'Psychologist'
    elif 'Therapist' in string: return 'Therapist'
    elif 'Counselor' in string: return 'Therapist'
simplifyTitle(title)

'Therapist'

In [161]:
name = page.find('h1',{'itemprop':'name'}).get_text()

def cleanText(string):
    cleanString = []
    stringparts = string.split(' ')
    for part in stringparts:
        if len(part) > 1 and '\n' not in part:
            cleanString.append(part)
    cleanString = ' '.join(cleanString)
    return cleanString

def cleanString(string):
    clean = ''
    for i in range(len(string)):
        if i == 0 and string[i] == ' ': pass
        elif string[i] == '\n': pass
        elif string[i] == ' ' and string[i-1] == ' ': pass
        elif string[i] == ' ' and string[i-1] == '\n': pass
        else: clean += string[i]
    return clean
        
cleanString(name)


'Catherine J. Adams'

In [94]:
x = 'gha      jhb'
x[3].isalpha()

False

In [155]:
def findRemote(pg):
# returns string; string may contain multiple methods of remote sessions eg online counseling and phone counseling
    if pg.find('div',{'class':'spec-list attributes-online-therapy'}):
        r = cleanString(pg.find('div',{'class':'spec-list attributes-online-therapy'}).get_text())
        return r.split('Video/Skype')[1]
    else:
        return 'Remote sessions not offered'
    
findRemote(page)

'Online Counseling'

In [178]:
insurancetext = mainpage.find('ul',{'class':'spec-list'}).get_text()

rawinsurancelist = insurancetext.split('\n')
insurancelist = []
for item in rawinsurancelist:
    if len(item) > 1 and item != 'More +' and item != 'Less -': insurancelist.append(item)

insurancedict = {'Out of Network':False}
for company in insurancelist:
    insurancedict[company] = False

insurancedict

{'Out of Network': False,
 'Any Insurance': False,
 'Cigna': False,
 'BlueCross and BlueShield': False,
 'Aetna': False,
 'Empire BlueCross BlueShield': False,
 '1199SEIU': False,
 'AMERIGROUP': False,
 'APS Healthcare': False,
 'Affinity Health Plan': False,
 'AllWays Health Partners': False,
 'Alliance': False,
 'AmeriHealth': False,
 'American Behavioral': False,
 'Anthem': False,
 'Beacon': False,
 'Beech Street': False,
 'Behavioral Health Systems': False,
 'Blue Care Network': False,
 'Blue Cross': False,
 'Blue Shield': False,
 'BlueCross BlueShield of Western New York': False,
 'BlueShield of Northeastern New York': False,
 'CCN': False,
 'CDPHP': False,
 "CHIP | Children's Program": False,
 'CHIPA': False,
 'CareConnect Health Plan': False,
 'CareFirst': False,
 'CareSource': False,
 'Cascade Health': False,
 'Ceridian': False,
 'ChoiceCare': False,
 'ComPsych': False,
 'Coventry': False,
 'E4Health': False,
 'EmblemHealth': False,
 'Excellus BlueCross BlueShield': False,
 'Fa

In [281]:
def findInsurance(pg,insdict):
    insurancestring = pg.find('div',{'class':'profile-finances details-section top-border'}).find('ul',{'class':'attribute-list copy-small'}).get_text()
    for company in insdict.keys():
        if company in insurancestring: insdict[company] = True
    return insdict
findInsurance(page,providers)

OrderedDict([('Any Insurance', 'False'),
             ('Cigna', True),
             ('BlueCross and BlueShield', True),
             ('Aetna', 'False'),
             ('Empire BlueCross BlueShield', True),
             ('1199SEIU', 'False'),
             ('AMERIGROUP', 'False'),
             ('APS Healthcare', 'False'),
             ('Affinity Health Plan', 'False'),
             ('AllWays Health Partners', 'False'),
             ('Alliance', 'False'),
             ('AmeriHealth', 'False'),
             ('American Behavioral', 'False'),
             ('Anthem', True),
             ('Beacon', 'False'),
             ('Beech Street', 'False'),
             ('Behavioral Health Systems', 'False'),
             ('Blue Care Network', 'False'),
             ('Blue Cross', 'False'),
             ('Blue Shield', 'False'),
             ('BlueCross BlueShield of Western New York', 'False'),
             ('BlueShield of Northeastern New York', 'False'),
             ('CCN', 'False'),
             ('C

In [280]:
import csv

# get list of insurance companies used
#insurancetext = mainpage.find('ul',{'class':'spec-list'}).get_text()
#rawinsurancelist = insurancetext.split('\n')
#insurancelist = []
#for item in rawinsurancelist:
#    if len(item) > 1 and item != 'More +' and item != 'Less -': insurancelist.append(item)

#insurancedict = {'Out of Network':False}
#for company in insurancelist:
#    insurancedict[company] = False
#insurancelist.append('Out of Network')
with open('providerlist.csv',mode='r') as csvfile:
    fieldnames = insurancelist
    reader = csv.DictReader(csvfile)
    for row in reader:
        providers = row
providers

OrderedDict([('Any Insurance', 'False'),
             ('Cigna', 'False'),
             ('BlueCross and BlueShield', 'False'),
             ('Aetna', 'False'),
             ('Empire BlueCross BlueShield', 'False'),
             ('1199SEIU', 'False'),
             ('AMERIGROUP', 'False'),
             ('APS Healthcare', 'False'),
             ('Affinity Health Plan', 'False'),
             ('AllWays Health Partners', 'False'),
             ('Alliance', 'False'),
             ('AmeriHealth', 'False'),
             ('American Behavioral', 'False'),
             ('Anthem', 'False'),
             ('Beacon', 'False'),
             ('Beech Street', 'False'),
             ('Behavioral Health Systems', 'False'),
             ('Blue Care Network', 'False'),
             ('Blue Cross', 'False'),
             ('Blue Shield', 'False'),
             ('BlueCross BlueShield of Western New York', 'False'),
             ('BlueShield of Northeastern New York', 'False'),
             ('CCN', 'False'),
    

In [340]:
#def getPsychData(city, state):

state = 'NY'
city = 'Brooklyn'

def getPageFromID(idnum):
    url = 'https://www.psychologytoday.com/profile/' + idnum
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15'}
    mainpagehtml = requests.get(url,headers=headers)
    return BeautifulSoup(mainpagehtml.content,'html.parser')


def getInfo(pg,insdict):
    gotInfo = {}
    
    gotInfo['name'] = cleanString(pg.find('h1',{'itemprop':'name'}).get_text())
    gotInfo['phone'] = pg.find('div',{'class':'container main-content profile'}).find('a').get('href').split('+')[1]
    gotInfo['address'] = pg.find('div',{'class':"address-data"}).get('data-address-rank-1')
    gotInfo['title'] = simplifyTitle(pg.find('div',{'class':'modal-subtitle'}).get_text())
    gotInfo['remote'] = findRemote(pg)
    if findInsurance(pg,insdict) is None: gotInfo['insurance'] = insdict
    else: gotInfo['insurance'] = findInsurance(pg,insdict)
    
    return gotInfo

def simplifyTitle(string):
    if 'Psychiatrist' in string: return "Psychiatrist"
    elif 'Psychologist' in string: return 'Psychologist'
    elif 'Therapist' in string: return 'Therapist'
    elif 'Counselor' in string: return 'Therapist'

def cleanString(string):
    clean = ''
    for i in range(len(string)):
        if i == 0 and string[i] == ' ': pass
        elif string[i] == '\n': pass
        elif string[i] == ' ' and string[i-1] == ' ': pass
        elif string[i] == ' ' and string[i-1] == '\n': pass
        else: clean += string[i]
    return clean

def findRemote(pg):
# returns string; string may contain multiple methods of remote sessions eg online counseling and phone counseling
    if pg.find('div',{'class':'spec-list attributes-online-therapy'}):
        r = cleanString(pg.find('div',{'class':'spec-list attributes-online-therapy'}).get_text())
        return r.split('Video/Skype')[1]
    else:
        return 'Remote sessions not offered'
    
def findInsurance(pg,insdict):
    insurances = pg.find('div',{'class':'profile-finances details-section top-border'}).find('ul',{'class':'attribute-list copy-small'})
    if insurances is None:
        return insdict
    else:
        insurancestring = insurances.get_text()
        for company in insdict.keys():
            if company in insurancestring: insdict[company] = True
        return insdict

In [285]:
url = 'https://www.psychologytoday.com/us/therapists/' + state.lower() + '/' + city.lower()
# headers added after running requests.get(url) by itself returned a 403 status code
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15'}
mainpagehtml = requests.get(url,headers=headers)

# turn html into BeautifulSoup object
soup = BeautifulSoup(mainpagehtml.content,'html.parser')
mainpage = soup.body

# site uses this div class for therapist listings in results. ResultSet object.
therapist_results = soup.find_all('div',{'class': 'result-row normal-result row'})

# get site ids for therapists taking new patients
therapist_site_ids = []
for t in therapist_results:
    if t.get('data-new-clients')=='1':
        therapist_site_ids.append(t.get('data-profid'))

In [287]:
# get dict of insurance providers from csv file created earlier by pulling providers from available filters on mainpage
# dict providers has insurance providers as keys and False for all values
with open('providerlist.csv',mode='r') as csvfile:
    fieldnames = insurancelist
    reader = csv.DictReader(csvfile)
    for row in reader:
        providers = row 

In [288]:
print(therapist_site_ids)

['703372', '441351', '81017', '342909', '92391', '198755', '444772', '31996', '86015', '198673', '388896', '488364', '290945', '251806', '83890', '409628', '468829']


In [345]:
# loop through ids to get page for each therapist        
#siteID = therapist_site_ids[10]
for siteID in therapist_site_ids:
    pggg = getPageFromID(siteID)
    print(getInfo(pggg,providers))
#insurances = pggg.find('div',{'class':'profile-finances details-section top-border'}).find('ul',{'class':'attribute-list copy-small'})
#len(insurances)

{'name': 'Ria Brown', 'phone': '1-929-295-5500', 'address': '300 Cadman Plaza West, Brooklyn, New York, 11201', 'title': 'Therapist', 'remote': 'Remote sessions not offered', 'insurance': OrderedDict([('Any Insurance', 'False'), ('Cigna', True), ('BlueCross and BlueShield', True), ('Aetna', True), ('Empire BlueCross BlueShield', True), ('1199SEIU', 'False'), ('AMERIGROUP', 'False'), ('APS Healthcare', 'False'), ('Affinity Health Plan', 'False'), ('AllWays Health Partners', 'False'), ('Alliance', 'False'), ('AmeriHealth', 'False'), ('American Behavioral', 'False'), ('Anthem', True), ('Beacon', True), ('Beech Street', 'False'), ('Behavioral Health Systems', 'False'), ('Blue Care Network', 'False'), ('Blue Cross', True), ('Blue Shield', True), ('BlueCross BlueShield of Western New York', 'False'), ('BlueShield of Northeastern New York', 'False'), ('CCN', 'False'), ('CDPHP', 'False'), ("CHIP | Children's Program", 'False'), ('CHIPA', 'False'), ('CareConnect Health Plan', 'False'), ('CareFi

{'name': 'Sara Walker-Peck', 'phone': '1-646-713-0550', 'address': '26 Court Street, Brooklyn, New York, 11242', 'title': 'Therapist', 'remote': 'Remote sessions not offered', 'insurance': OrderedDict([('Any Insurance', 'False'), ('Cigna', True), ('BlueCross and BlueShield', True), ('Aetna', True), ('Empire BlueCross BlueShield', True), ('1199SEIU', 'False'), ('AMERIGROUP', 'False'), ('APS Healthcare', 'False'), ('Affinity Health Plan', 'False'), ('AllWays Health Partners', 'False'), ('Alliance', 'False'), ('AmeriHealth', 'False'), ('American Behavioral', 'False'), ('Anthem', True), ('Beacon', True), ('Beech Street', 'False'), ('Behavioral Health Systems', 'False'), ('Blue Care Network', 'False'), ('Blue Cross', True), ('Blue Shield', True), ('BlueCross BlueShield of Western New York', 'False'), ('BlueShield of Northeastern New York', 'False'), ('CCN', 'False'), ('CDPHP', 'False'), ("CHIP | Children's Program", 'False'), ('CHIPA', 'False'), ('CareConnect Health Plan', 'False'), ('CareF

{'name': 'Jen Wittlin', 'phone': '1-929-203-8143', 'address': None, 'title': 'Therapist', 'remote': 'Remote sessions not offered', 'insurance': OrderedDict([('Any Insurance', 'False'), ('Cigna', True), ('BlueCross and BlueShield', True), ('Aetna', True), ('Empire BlueCross BlueShield', True), ('1199SEIU', 'False'), ('AMERIGROUP', 'False'), ('APS Healthcare', 'False'), ('Affinity Health Plan', 'False'), ('AllWays Health Partners', 'False'), ('Alliance', 'False'), ('AmeriHealth', 'False'), ('American Behavioral', 'False'), ('Anthem', True), ('Beacon', True), ('Beech Street', 'False'), ('Behavioral Health Systems', 'False'), ('Blue Care Network', 'False'), ('Blue Cross', True), ('Blue Shield', True), ('BlueCross BlueShield of Western New York', 'False'), ('BlueShield of Northeastern New York', 'False'), ('CCN', 'False'), ('CDPHP', 'False'), ("CHIP | Children's Program", 'False'), ('CHIPA', 'False'), ('CareConnect Health Plan', 'False'), ('CareFirst', 'False'), ('CareSource', 'False'), ('C

{'name': 'Sara Schulman', 'phone': '1-917-746-8480', 'address': '1331 East 16th Street, Brooklyn, New York, 11230', 'title': 'Therapist', 'remote': 'Remote sessions not offered', 'insurance': OrderedDict([('Any Insurance', 'False'), ('Cigna', True), ('BlueCross and BlueShield', True), ('Aetna', True), ('Empire BlueCross BlueShield', True), ('1199SEIU', 'False'), ('AMERIGROUP', 'False'), ('APS Healthcare', 'False'), ('Affinity Health Plan', 'False'), ('AllWays Health Partners', 'False'), ('Alliance', 'False'), ('AmeriHealth', 'False'), ('American Behavioral', 'False'), ('Anthem', True), ('Beacon', True), ('Beech Street', 'False'), ('Behavioral Health Systems', 'False'), ('Blue Care Network', 'False'), ('Blue Cross', True), ('Blue Shield', True), ('BlueCross BlueShield of Western New York', 'False'), ('BlueShield of Northeastern New York', 'False'), ('CCN', 'False'), ('CDPHP', 'False'), ("CHIP | Children's Program", 'False'), ('CHIPA', 'False'), ('CareConnect Health Plan', 'False'), ('Ca

{'name': 'Sarah A Nadeau', 'phone': '1-646-576-7424', 'address': '95-11 Shore Rd, Brooklyn, New York, 11209', 'title': 'Therapist', 'remote': 'Remote sessions not offered', 'insurance': OrderedDict([('Any Insurance', 'False'), ('Cigna', True), ('BlueCross and BlueShield', True), ('Aetna', True), ('Empire BlueCross BlueShield', True), ('1199SEIU', 'False'), ('AMERIGROUP', 'False'), ('APS Healthcare', 'False'), ('Affinity Health Plan', 'False'), ('AllWays Health Partners', 'False'), ('Alliance', 'False'), ('AmeriHealth', 'False'), ('American Behavioral', 'False'), ('Anthem', True), ('Beacon', True), ('Beech Street', 'False'), ('Behavioral Health Systems', 'False'), ('Blue Care Network', 'False'), ('Blue Cross', True), ('Blue Shield', True), ('BlueCross BlueShield of Western New York', 'False'), ('BlueShield of Northeastern New York', 'False'), ('CCN', 'False'), ('CDPHP', 'False'), ("CHIP | Children's Program", 'False'), ('CHIPA', 'False'), ('CareConnect Health Plan', 'False'), ('CareFirs

{'name': 'Zena Watson', 'phone': '1-929-203-5361', 'address': None, 'title': 'Therapist', 'remote': 'Online Counseling', 'insurance': OrderedDict([('Any Insurance', 'False'), ('Cigna', True), ('BlueCross and BlueShield', True), ('Aetna', True), ('Empire BlueCross BlueShield', True), ('1199SEIU', 'False'), ('AMERIGROUP', 'False'), ('APS Healthcare', 'False'), ('Affinity Health Plan', 'False'), ('AllWays Health Partners', 'False'), ('Alliance', 'False'), ('AmeriHealth', 'False'), ('American Behavioral', 'False'), ('Anthem', True), ('Beacon', True), ('Beech Street', 'False'), ('Behavioral Health Systems', 'False'), ('Blue Care Network', 'False'), ('Blue Cross', True), ('Blue Shield', True), ('BlueCross BlueShield of Western New York', 'False'), ('BlueShield of Northeastern New York', 'False'), ('CCN', 'False'), ('CDPHP', 'False'), ("CHIP | Children's Program", 'False'), ('CHIPA', 'False'), ('CareConnect Health Plan', 'False'), ('CareFirst', 'False'), ('CareSource', 'False'), ('Cascade Hea

['HERE', 'a', 'HERE', 'z']